# Review Classification

## Batching and Model Training

### Import Libraries

In [1]:
import pandas as pd # Loading data
import numpy as np
import warnings 
import re # text matching
from collections import Counter # for vocabulary
from sklearn.model_selection import train_test_split # train test splits

warnings.filterwarnings('ignore')

### Data Loading and Processing

We will first do all the necessary pre-processing before starting to create batches and training the model. All the steps are explained in the notebook named `Text Cleaning.ipynb`

In [2]:
# Read dataset
data = pd.read_csv("Reviews.csv")
# Drop unnecesary columns and duplicates
new_data = data.drop_duplicates(subset=['UserId', 'ProfileName', 'Time', 'Text'])
# Get useful columns
useful_data = new_data[['Text', 'Score']]
# Calculate length of each sentence without tokenizer
useful_data['sudo_length'] = useful_data.Text.str.split().str.len()
# Filter examples by length
useful_data = useful_data[(useful_data.sudo_length > 20) & (useful_data.sudo_length < 100)]
# Remove length column
useful_data = useful_data.drop(['sudo_length'], axis = 1)
# print 5 rows
useful_data.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


### Tokenizing and Creating vocabulary
Now its time to tokenize and create our vocabulary. We use the `TextProcessor` class on data splits.

In [3]:
class TextProcessor:
    def __init__(self):
        self.vocab_dict = dict({"<unk>" : 0, "<pad>" : 1})
        self.counter = Counter()

    def tokenize(self, sent):
        if sent.endswith("."):
            sent = sent[:-1]
        new_x = re.sub('<.*?>', ' ', sent)
        new_x = re.sub('\s\s+',' ', new_x)
        new_x = re.sub('\W\s', ' ', new_x)
        new_x = re.sub('\w\W{2,}', ' ', new_x)
        new_x = new_x.lower().split()
        return new_x
        
    def processDataset(self, sent):
        tokens = self.tokenize(sent)
        token_set = set(tokens)
        self.counter.update(Counter(tokens))
        return len(tokens)
        
    def build_vocab(self, num_most_common_to_use=10000):
        words = self.counter.most_common(num_most_common_to_use)
        for i in range(num_most_common_to_use - 2):
            self.vocab_dict[words[i][0]] = len(self.vocab_dict)
            
    def tokenize_and_return_length(self, sent):
        tokens = self.tokenize(sent)
        return len(tokens)
            
    def process(self, sent):
        tokens = self.tokenize(sent)
        processed = []
        for val in tokens:
            processed.append(self.vocab_dict.get(val, self.vocab_dict["<unk>"]))
            
        return processed

#### Create Train and Test sets

In [4]:
train, test = train_test_split(useful_data, test_size = 0.2)

Run text processor to create vocabulary. Also create a new column denoting length of tokens for corresponding review. This will be used in creating batches.

In [5]:
textprocessor = TextProcessor()
train['length'] = train.Text.apply(textprocessor.processDataset)
textprocessor.build_vocab(40000)

train.head()

,Text,Score,length
110566,A ship fast. Safely arrived. Starbucks Via Col...,5,31
254509,This product arrived completely melted. Rather...,3,40
60660,"I tried this for the first time today, but I o...",1,26
461159,"Seriously, how can one think of making risotto...",5,26
3299,"great deal, especially for the price. Item arr...",5,44


In [6]:
test['length'] = test.Text.apply(textprocessor.tokenize_and_return_length)

test.head()

,Text,Score,length
335596,"This tea isn't bad, it just made (to me) a wea...",2,68
209603,"Duke, the 17 year old wonderdog, gives these f...",4,90
429106,My kids love this stuff and it is perfect for ...,5,54
487595,"There are other imitators out there, but none ...",5,74
84721,This black type of cardamom is impossible to f...,5,56


### Batching and Data Loader creation
We are going to use `PyTorch` for training an `LSTM Model` for classification of reviews. Before creating the model, we first need to create dataloader, so that we can conviniently pass our training and testing examples to our model.

First we will create a *Custom PyTorch* dataset class which will preprocess our examples and convert them into a set of indices corresponging to vocabulary we just created.

In [7]:
import torch
from torch.utils.data import Dataset

In [8]:
class ReviewDataset(Dataset):
    def __init__(self, df, processor):
        self.data = df
        self.data = self.data.sort_values(by='length')
        self.tprocess = processor
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row.Text
        label = row.Score
        
        text = self.tprocess.process(text)
        
        return (text, label)

As you can observe that our dataset class sorted our dataframe by the length of each sentence. This allows us to create a batch with minmum padding, as we will see later when creating batches.

In [9]:
train_dataset = ReviewDataset(train, textprocessor)
test_dataset = ReviewDataset(test, textprocessor)

print("Sample Data : ")
print(train_dataset[0])

Sample Data : 
([66, 0, 81, 14844], 5)


### Creating Dataloader

#### Import Dataloader class

In [10]:
from torch.utils.data import DataLoader

#### Custom batch formation class
Since our dataset contains all the examples in sorted fashion, the batch we will get from our dataloader will have the largest length sentence at the end of the batch list. In the batch collator class, we will first create an array or size `(batch_size, seq_len)`, where seq_len will be equal to the length of last sentence recieved in batch.

As all the examples are sorted, padding required within a batch will be minimum as nearly equal length examples will be sampled.

In [11]:
class MyCollator(object):
    def __init__(self, pad_token = 1):
        self.pad = pad_token
    def __call__(self, batch):
        batch_size = len(batch)
        seq_len = len(batch[-1][0])
        formed = np.zeros((batch_size, seq_len), dtype = np.long) + self.pad
        labels = []
        for i in range(batch_size):
            example = batch[i]
            formed[i, :len(example[0])] = example[0]
            labels.append(example[1])
            
        return torch.LongTensor(formed), torch.LongTensor(labels)

In [12]:
BATCH_SIZE = 64

collator = MyCollator()
dloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collator)

#### Example batch

In [13]:
batch = next(iter(dloader))
print("Examples : ")
print(batch[0])
print("Labels : ")
print(batch[1])

Examples : 
tensor([[   66,     0,    81,  ...,     1,     1,     1],
        [  384,    56,   207,  ...,     1,     1,     1],
        [    0,   111,   376,  ...,     1,     1,     1],
        ...,
        [ 2077,    13,   294,  ..., 34658,   865, 34141],
        [   29,   123,     7,  ...,   100,    64,  1648],
        [    3,    15,  4826,  ...,   198,    10,   462]])
Labels : 
tensor([5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 4, 1, 5, 5, 1, 5, 5, 5, 1, 5, 5, 2, 5,
        5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 1, 5, 4, 5, 1, 4, 5, 4, 5, 5, 5])


### Calculating calss weights

In [14]:
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight('balanced', sorted(train.Score.unique()), train.Score)
for cat in sorted(train.Score.unique()):
    print("Class {} : {:.5f}".format(cat, weights[cat - 1]))

Class 1 : 2.16674
Class 2 : 4.01726
Class 3 : 2.93625
Class 4 : 1.54014
Class 5 : 0.30306


Now we are ready to start creating our model for Classification!!

### Import Pytorch and other modules

In [15]:
import torch.nn as nn
import torch.optim as opt
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import copy

torch.manual_seed(101)

#### Creating Base Model
The base model is the simplest model we train. It consist of 
- **Embedding layer** : this layer transforms our indexes to 300-D vector that respresent a word.
- **LSTM cell** : the core of our model, the LSTM cell
- **Linear layer** : we take the last output of the lstm layer and use it to predict classes

In [16]:
class BaseModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(BaseModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_size = hidden_size
        self.cell = nn.LSTM(embedding_dim, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, num_classes)
        self.soft = nn.Softmax(dim=1)
        
    def forward(self, x, hstate = None):
        if hstate is None:
            hstate = self.init_hidden(self.hidden_size, x.shape[0])
            
        cell_out, _ = self.cell(self.embedding(x), hstate)
        
        out = self.linear(cell_out[:, -1, :])
        
        return self.soft(out)
            
    def init_hidden(self, hidden_size, bs):
        return (torch.zeros(1, bs, hidden_size, device=device), torch.zeros(1, bs, hidden_size, device=device))

#### Creating evalutaion metrics
The evaluation metrics for classification used.

In [17]:
class ClassificationMetrics:
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.classes = list(range(num_classes))
        self.epsilon = 1e-12
        self.cmatrix = np.zeros((num_classes, num_classes), dtype = np.int64) + self.epsilon
        
        self.total_correct = 0
        self.total_examples = 0
        
    def update(self, pred, truth):
        pred = pred.cpu()
        truth = truth.cpu()
        
        _, idx = pred.topk(1)
        truth = truth.view(-1, 1)
        
        self.total_examples += len(truth)
        self.total_correct += sum(idx == truth).item()
        
        val = confusion_matrix(truth, idx, labels=self.classes)
        
        self.cmatrix = self.cmatrix + val
        
        
    def precision_score(self):
        scores = {}
        for i in range(self.num_classes):
            scores[i] = self.cmatrix[i, i] / (sum(self.cmatrix[:, i]) + self.epsilon)
        
        return scores
    
    def recall_score(self):
        scores = {}
        for i in range(self.num_classes):
            scores[i] = self.cmatrix[i, i] / (sum(self.cmatrix[i, :]) + self.epsilon)
        
        return scores
    
    def scores(self, return_type = 'f1'):
        pscores = self.precision_score()
        rscores = self.recall_score()
        scores = {}
        for i in range(self.num_classes):
            if(pscores[i] == 0 and rscores[i] == 0):
                scores[i] = 0
            else:
                scores[i] = 2 * ((pscores[i] * rscores[i]) / (pscores[i] + rscores[i])  + self.epsilon)
            
        if return_type == 'f1':
            return scores
        elif return_type == 'all':
            all_scores = list(zip(pscores.values(), rscores.values(), scores.values()))
            t = {}
            for i in range(self.num_classes):
                t[i] = all_scores[i]
                
            return t
        else:
            raise Exception("Invalid argument for return type")
            
    def accuracy_score(self):
        return self.total_correct / self.total_examples
    
    def reset(self):
        self.total_correct = 0
        self.total_examples = 0
        self.cmatrix = np.zeros((self.num_classes, self.num_classes))
            
    def print_report(self):
        all_scores = self.scores('all')
        print("{:^15}\t{:^15}\t{:^15}\t{:^15}".format("Class", "Precision", "Recall", "F1-score"))
        for c, values in all_scores.items():
            print("{:^15}\t{:^15.3f}\t{:^15.3f}\t{:^15.3f}".format(c, values[0], values[1], values[2]))
            
        print("Accuracy : {:.5f} %".format(self.accuracy_score()))

Creating necessary variables along with our BaseModel, loss function and optimizer.

In [18]:
VOCAB_SIZE = len(textprocessor.vocab_dict)
HIDDEN_SIZE = 128
EMB_DIM = 200
NUM_CLASSES = 5
device = 'cuda'

net = BaseModel(VOCAB_SIZE, EMB_DIM, HIDDEN_SIZE, NUM_CLASSES)
net = net.cuda()
print(net)

metrics = ClassificationMetrics(NUM_CLASSES)
test_metrics = ClassificationMetrics(NUM_CLASSES)

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))
optim = opt.Adam(net.parameters(), lr = 0.001)

BaseModel(
  (embedding): Embedding(40000, 200)
  (cell): LSTM(200, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=5, bias=True)
  (soft): Softmax(dim=1)
)


#### Training loop

In [ ]:
N_EPOCHS = 10

pltloss = []
pltacc = []

pltloss_test = []
pltacc_test = []

best_val_acc = 0
best_model = copy.deepcopy(net)

for epoch in range(N_EPOCHS):
    losses = []
    net.train()
    print("Training Loop")
    for batch in tqdm(dloader):
        metrics.reset()
        optim.zero_grad()

        X, labels = batch[0].to(device), (batch[1] - 1).to(device)
        pred = net(X)
        loss = criterion(pred, labels)
        losses.append(loss.item())
        loss.backward()
        optim.step()
        metrics.update(pred, labels)
        
    print("Training Run\nEpoch : {} Loss : {:.5f}".format(epoch + 1, sum(losses) / len(losses)))
    metrics.print_report()
    pltloss.append(sum(losses) / len(losses))
    pltacc.append(metrics.accuracy_score() * 100)
    
    test_losses = []
    net.eval()
    print("Validation Loop")
    for batch in tqdm(test_loader):
        test_metrics.reset()

        X, labels = batch[0].to(device), (batch[1] - 1).to(device)
        pred = net(X)
        loss = criterion(pred, labels)
        test_losses.append(loss.item())
        
        test_metrics.update(pred, labels)
        
    print("Validation Run\nEpoch : {} Loss : {:.5f}".format(epoch + 1, sum(test_losses) / len(test_losses)))
    test_metrics.print_report()
    pltloss_test.append(sum(test_losses) / len(test_losses))
    pltacc_test.append(test_metrics.accuracy_score() * 100)
    
    if((test_metrics.accuracy_score() * 100) > best_val_acc):
        best_val_acc = test_metrics.accuracy_score() * 100
        best_model = copy.deepcopy(net)

  0%|                                                                                         | 0/3506 [00:00<?, ?it/s]

Training Loop


  0%|▎                                                                                 | 3/877 [00:00<00:39, 22.28it/s]

Training Run
Epoch : 1 Loss : 1.46223
     Class     	   Precision   	    Recall     	   F1-score    
       0       	     0.000     	     0.000     	     0.000     
       1       	     0.000     	     0.000     	     0.000     
       2       	     0.000     	     0.000     	     0.000     
       3       	     0.250     	     0.667     	     0.364     
       4       	     0.733     	     0.647     	     0.688     
Accuracy : 0.54167 %
Validation Loop


  0%|                                                                                 | 3/3506 [00:00<02:08, 27.35it/s]

...

  0%|                                                                                 | 3/3506 [00:00<02:15, 25.93it/s]

Validation Run
Epoch : 9 Loss : 1.37612
     Class     	   Precision   	    Recall     	   F1-score    
       0       	     1.000     	     0.333     	     0.500     
       1       	     0.000     	     0.000     	     0.000     
       2       	     0.143     	     1.000     	     0.250     
       3       	     0.500     	     0.500     	     0.500     
       4       	     1.000     	     0.692     	     0.818     
Accuracy : 0.59091 %
Training Loop


  1%|▍                                                                                 | 5/877 [00:00<00:18, 46.42it/s]

Training Run
Epoch : 10 Loss : 1.16154
     Class     	   Precision   	    Recall     	   F1-score    
       0       	     0.000     	     0.000     	     0.000     
       1       	     0.333     	     0.500     	     0.400     
       2       	     0.200     	     0.500     	     0.286     
       3       	     0.500     	     0.667     	     0.571     
       4       	     1.000     	     0.647     	     0.786     
Accuracy : 0.62500 %
Validation Loop


 48%|██████████████████████████████████████▋                                         | 424/877 [00:11<00:15, 28.58it/s]

### Ploting loss curve

In [ ]:
plt.plot(list(range(len(pltloss))), pltloss)
plt.plot(list(range(len(pltloss_test))), pltloss_test)

plt.title("Loss")

### Ploting Accuracy curve

In [ ]:
plt.plot(list(range(len(pltacc))), pltacc, )
plt.plot(list(range(len(pltacc_test))), pltacc_test)

plt.title("Accuracy")